In [1]:
from XMTR import MTR
import numpy as np
import pandas as pd
from scipy.io import arff
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [6]:
def calc_al_error(instance, perc):
  # the error should be non zero
  return (abs(instance)+0.1)*perc 

fb_df = pd.read_csv('dataset_Facebook.csv', sep=';')#, nrows=200)

# fill NaN
fb_df['like'].fillna(0,inplace=True)
fb_df['share'].fillna(0,inplace=True)
fb_df['Paid'].fillna(0,inplace=True)
fb_df.drop(['Type'], inplace=True, axis=1)


# get column names
column_names = fb_df.columns

# get data/target names
f_n = column_names[:14]
#t_n = ['comment', 'share']
t_n = column_names[14:]

X = fb_df[f_n]
y = fb_df[t_n]

# convert to numpy
X = X.to_numpy()
y = y.to_numpy()

scaler = MinMaxScaler(feature_range=(0,10))
y = scaler.fit_transform(y)

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=42)
allowed_error = 0.5

max_ds = [2, 5, 10]
min_s_ls = [5, 10]
m_fs = ['sqrt', 0.75, None]
ests = [100, 500, 1000]
total = (len(max_ds)*len(min_s_ls)*len(m_fs)*len(ests))
counter = 1
min_error = 10**10 # just a ref
for max_d in max_ds:
  for min_s_l in min_s_ls:
    for m_f in m_fs:
      for est in ests:
        # create/train model
        RFmodel = RandomForestRegressor(n_estimators=est, max_depth=max_d, min_samples_leaf=min_s_l, max_features=m_f, n_jobs=-1, random_state=42)
        RFmodel.fit(X_train, y_train)
        MTR_obj = MTR(model=RFmodel, X_train = X_train, X_test=X_test, y_train=y_train, y_test=y_test, feature_names=f_n, target_names=t_n)
        #avgRuleLengths = 0
        avgEstimators = []
        #actualpreds = []
        MTRpreds = []
        # get results
        for i in range(len(X_test)):
          error = calc_al_error(y_test[i], allowed_error)
          rule = MTR_obj.explain(X_test[i], error) # explain instance
          
          #actualpreds.append(y_test[i])
          decisionsAndErrors = MTR_obj.getDecisionsAndErros() # get preds/errors
          MTRpreds.append(decisionsAndErrors)

          #feature_limits = MTR_obj.getFeatureLimits()
          #avgRuleLengths += len(feature_limits.keys())
          estimators = MTR_obj.getAmountOfReduction() # get estimators
          avgEstimators.append(estimators[0])
        #finalLen = avgRuleLengths/len(X_test)

        #actualpreds = np.array(actualpreds)
        MTRpreds = np.array(MTRpreds)

        # local error
        MTRerrors = np.array([subarray[:,1] for subarray in MTRpreds])
        total_error = np.mean(MTRerrors, axis=0) # error per target
        final_error = np.array(total_error).mean() # avg error

        if final_error < min_error:
          min_error = final_error
          best_par = {"max_d=":max_d, "min_s_l=":min_s_l, "m_f=": m_f, "est=": est, "local error=": final_error}
        print(counter, '/', total, "| max_depth=",max_d," | ", "min_sample_leaf=",min_s_l," | ", "max_f=", m_f," | ", "est=", est," | ", "local error=", final_error, " | ", "estimators=", round(np.array(avgEstimators).mean(),3),"/",est)
        counter += 1

1 / 54 | max_depth= 2  |  min_sample_leaf= 5  |  max_f= sqrt  |  est= 100  |  local error= 0.0  |  estimators= 100.0 / 100
2 / 54 | max_depth= 2  |  min_sample_leaf= 5  |  max_f= sqrt  |  est= 500  |  local error= 0.0  |  estimators= 500.0 / 500
3 / 54 | max_depth= 2  |  min_sample_leaf= 5  |  max_f= sqrt  |  est= 1000  |  local error= 0.0  |  estimators= 1000.0 / 1000
4 / 54 | max_depth= 2  |  min_sample_leaf= 5  |  max_f= 0.75  |  est= 100  |  local error= 0.0  |  estimators= 100.0 / 100
5 / 54 | max_depth= 2  |  min_sample_leaf= 5  |  max_f= 0.75  |  est= 500  |  local error= 0.0  |  estimators= 500.0 / 500
6 / 54 | max_depth= 2  |  min_sample_leaf= 5  |  max_f= 0.75  |  est= 1000  |  local error= 0.0  |  estimators= 1000.0 / 1000
7 / 54 | max_depth= 2  |  min_sample_leaf= 5  |  max_f= None  |  est= 100  |  local error= 0.013587084720714654  |  estimators= 99.44 / 100
8 / 54 | max_depth= 2  |  min_sample_leaf= 5  |  max_f= None  |  est= 500  |  local error= 0.014276188907102435  |  

In [7]:
best_par

{'max_d=': 2, 'min_s_l=': 5, 'm_f=': 'sqrt', 'est=': 100, 'local error=': 0.0}

In [ ]:
# WORST
#18 / 54 | max_depth= 2  |  min_sample_leaf= 10  |  max_f= None  |  est= 1000  |  local error= 0.05122241632612199  |  estimators= 964.14 / 1000